In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,375 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,125 kB]
Fetched 3,732 kB in 3s (1,386 kB/s)
Reading package lists... Do

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [5]:
# Used this cell as alternative data pull, errors with suggested method

# Add file to SparkFiles
spark.sparkContext.addFile("https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv")

# Read into DataFrame
homesales_df = spark.read.csv("file://" + SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)

In [4]:
# 1. Read in the AWS S3 bucket into a DataFrame. ## Used cell above to add locally and read in^^^
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

homesales_df = spark.read.csv(url, header=True, inferSchema=True)

UnsupportedOperationException: None

In [11]:
homesales_df

DataFrame[id: string, date: date, date_built: int, price: int, bedrooms: int, bathrooms: int, sqft_living: int, sqft_lot: int, floors: int, waterfront: int, view: int]

In [7]:
# 2. Create a temporary view of the DataFrame.
homesales_df.createOrReplaceTempView("homesales_view")


In [12]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
# Write a query to produce result selecting avg price (rounded) from the view created if the home has 4 bedrooms
result = spark.sql("""
    SELECT YEAR(date) AS Year,
           ROUND(AVG(price), 2) AS Average_Price
    FROM home_sales_view
    WHERE bedrooms = 4
    GROUP BY YEAR(date)
    ORDER BY Year
""")

result.show()



+----+-------------+
|Year|Average_Price|
+----+-------------+
|2019|     300263.7|
|2020|    298353.78|
|2021|    301819.44|
|2022|    296363.88|
+----+-------------+



In [13]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?

result01 = spark.sql("""
    SELECT date_built AS Year_Built,
           ROUND(AVG(price), 2) AS Average_Price
    FROM home_sales_view
    WHERE bedrooms = 3 AND bathrooms = 3
    GROUP BY date_built
    ORDER BY date_built
""")

# Show the result
result01.show()


+----------+-------------+
|Year_Built|Average_Price|
+----------+-------------+
|      2010|    292859.62|
|      2011|    291117.47|
|      2012|    293683.19|
|      2013|    295962.27|
|      2014|    290852.27|
|      2015|     288770.3|
|      2016|    290555.07|
|      2017|    292676.79|
+----------+-------------+



In [14]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?

result02 = spark.sql("""
    SELECT date_built AS Year_Built,
           ROUND(AVG(price), 2) AS Average_Price
    FROM home_sales_view
    WHERE bedrooms = 3
          AND bathrooms = 3
          AND floors = 2
          AND sqft_living >= 2000
    GROUP BY date_built
    ORDER BY date_built
""")

# Show the result
result02.show()


+----------+-------------+
|Year_Built|Average_Price|
+----------+-------------+
|      2010|    285010.22|
|      2011|    276553.81|
|      2012|    307539.97|
|      2013|    303676.79|
|      2014|    298264.72|
|      2015|    297609.97|
|      2016|     293965.1|
|      2017|    280317.58|
+----------+-------------+



In [15]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

# Adding run time counter to query to display result and runtime below
start_time = time.time()

result03 = spark.sql("""
    SELECT view,
           ROUND(AVG(price), 2) AS Average_Price
    FROM home_sales_view
    GROUP BY view
    HAVING AVG(price) >= 350000
    ORDER BY view DESC
""")


result03.show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+-------------+
|view|Average_Price|
+----+-------------+
| 100|    1026669.5|
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows

--- 0.9549338817596436 seconds ---


In [17]:
# 7. Cache the the temporary table home_sales.

spark.catalog.cacheTable("homesales_view")


In [18]:
# 8. Check if the table is cached.
spark.catalog.isCached('homesales_view')

True

In [19]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()


result04 = spark.sql("""
    SELECT view,
           ROUND(AVG(price), 2) AS Average_Price
    FROM homesales_view
    GROUP BY view
    HAVING AVG(price) >= 350000
    ORDER BY view DESC
""")

# Show the result
result04.show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+-------------+
|view|Average_Price|
+----+-------------+
| 100|    1026669.5|
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows

--- 1.9767940044403076 seconds ---


In [23]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
# added a path variable that uses file due to trouble with file path reading
output_path = "file:///C:/Users/trist/OneDrive/Desktop/mod22"
homesales_df.write.partitionBy("date_built").parquet(output_path)


In [24]:
# 11. Read the parquet formatted data.

parquet_df = spark.read.parquet("file:///C:/Users/trist/OneDrive/Desktop/mod22")


In [25]:
# 12. Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView("parquet_data")

In [28]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

from pyspark.sql.functions import round, avg

start_time = time.time()

# Writing query w python syntax on parquet to avoid error from previous
result05 = parquet_df.groupBy("view").agg(round(avg("price"), 2).alias("Average_Price")) \
                   .filter("Average_Price >= 350000") \
                   .orderBy("view", ascending=False)

# Show the result
result05.show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+-------------+
|view|Average_Price|
+----+-------------+
| 100|    1026669.5|
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows

--- 0.9384922981262207 seconds ---


In [29]:
# 14. Uncache the home_sales temporary table.

spark.catalog.uncacheTable("homesales_view")


In [30]:
# 15. Check if the home_sales is no longer cached

spark.catalog.isCached("homesales_view")

False